In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import wget
import os
from datetime import datetime
from random import randrange, randint
from datetime import timedelta
import json
import re

In [11]:
# Starting from the top
print('Starting crawling script.')

Starting crawling script.


In [12]:
# Get recipe categories
base = 'https://www.myfoodandfamily.com/recipes/'
pages = {'dinner': [3, 4, 6, 8, 7, 5],
        'regional': [412, 202, 118, 486, 372, 30, 362, 140, 149, 271],
        'holiday': [597, 10, 21],
        }

In [13]:
# want category, subcategory, and link
data = []

base = 'https://www.myfoodandfamily.com/recipes/'
for category in pages:
    subcategories = pages[category]
    for subcategoryid in subcategories:
        print('Scraping category: %s on id #%i' % (category, subcategoryid))
        url = base + str(subcategoryid)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        subcategoryname = soup.find('h1').text
        cards = soup.find_all('div', class_='cls_outerCard item-card')
        for card in cards:
            link = 'https://www.myfoodandfamily.com/' + card.find('a')['href']
            image = card.find('img')['data-yo-src']
            tup = (category, subcategoryname, link, image)
            data.append(tup)

Scraping category: dinner on id #3
Scraping category: dinner on id #4
Scraping category: dinner on id #6
Scraping category: dinner on id #8
Scraping category: dinner on id #7
Scraping category: dinner on id #5
Scraping category: regional on id #412
Scraping category: regional on id #202
Scraping category: regional on id #118
Scraping category: regional on id #486
Scraping category: regional on id #372
Scraping category: regional on id #30
Scraping category: regional on id #362
Scraping category: regional on id #140
Scraping category: regional on id #149
Scraping category: regional on id #271
Scraping category: holiday on id #597
Scraping category: holiday on id #10
Scraping category: holiday on id #21


In [14]:
print('There are %i recipes' % len(data))

There are 430 recipes


In [ ]:
totalData = []
for index, recipe in enumerate(data):
    if index % 20 == 0:
        print('On index %i' % index)
    category, subcategory, link, imageLink = recipe
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('h1').text.strip()
    
    timeTag = soup.find('div', attrs = {'data-qa': 'qa-cook-timer'})
    if timeTag is not None:
        estimatedTime = timeTag.text.strip()
    else:
        estimatedTime = None
        
    descTag = soup.find('div', attrs = {'data-qa':'qa-prd-desc'})
    if descTag is not None:
        description = descTag.text.strip()
    else:
        description = None
    
    ingredients = []
    ingredientsList = soup.find_all('div', attrs = {'data-qa':'qa-ingreList'})
    for element in ingredientsList:
        ingredients.append(element.text.strip())
        
    procedure = []
    procedureContainer = soup.find('div', attrs = {'data-qa':'qa-recipe-make-details'})
    procedureList = procedureContainer.find_all('div', class_='krRecipeDirectionsDest')
    for step in procedureList:
        # [2:] to get rid of number
        procedure.append(step.text[2:].strip())
        
    # Don't download if already exists
    if imageLink is not None:
        downloadLocation = f'./src/recipeImages/{title}.jpg'
        if not os.path.exists(downloadLocation):
            if imageLink.startswith('//'):
                imageLink = 'https:' + imageLink
                myfile = requests.get(imageLink)
                open(downloadLocation, 'wb').write(myfile.content)
            else:
                wget.download(imageLink, downloadLocation)
        imageName = title + '.jpg'
    else:
        imaegName = None
    
    recipeObject = {
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'description': description,
        'ingredients': ingredients,
        'procedure': procedure,
        'imageName': imageName,
        'estimatedTime': estimatedTime
    }
    totalData.append(recipeObject)
        

On index 0
./src/recipeImages/Baked Crab Rangoon.jpg
./src/recipeImages/Country-Style Smashed Potato Salad.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/125455_640x428.jpg
./src/recipeImages/Hearty Italian Sausage Soup.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/HeartyItallianSSausageSoup_01.jpg?id=375b0eaa2fda971dfbc16075522e252b8401eef7&ht=225&wd=225&yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/HeartyItallianSSausageSoup_01.jpg?id=375b0eaa2fda971dfbc16075522e252b8401eef7&ht=225&wd=225&yocs=1b_
./src/recipeImages/Hearty Italian Sausage Soup.jpg
./src/recipeImages/Lentil Soup.jpg
made it
//assets.kraftfoods.com/recipe_images/Lentil-Soup-43434.jpg
./src/recipeImages/Speedy Barbecue Beef Stir-Fry.jpg
made it
//assets.kr

./src/recipeImages/Vanilla-Berry Trifle in a Cloud.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/Vanilla-Berry_Trifle_in_a_Cloud_640x428.jpg
./src/recipeImages/Banana Split Poke Cake.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/19b478d02d150137fdeb123dfe2baf36.yottaa.net/v~4b.179/khassets/ugc/recipeimages/73aa9427-cfe2-4372-a0e5-24fc81b1e6a8_290x194.jpg?yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/19b478d02d150137fdeb123dfe2baf36.yottaa.net/v~4b.179/khassets/ugc/recipeimages/73aa9427-cfe2-4372-a0e5-24fc81b1e6a8_290x194.jpg?yocs=1b_
./src/recipeImages/Banana Split Poke Cake.jpg
./src/recipeImages/Bears on Cloud-Nine.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/121453_640x428.jpg
./src/recipeImages/Caribbean Fudge Pie.jpg
made it
//assets.kraftfoods.com/recipe_images/Caribbean_Fudge_Pie.jpg
./src/recipeImages/Cookie Butter Cheesecake.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3

./src/recipeImages/Cheesy Chicken Ranch Sandwiches.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/57487_3000x2000.jpg?id=5e7df3153c3fea306e8d2b0c496646510f8fe485&ht=225&wd=225&clid=pim&yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/57487_3000x2000.jpg?id=5e7df3153c3fea306e8d2b0c496646510f8fe485&ht=225&wd=225&clid=pim&yocs=1b_
./src/recipeImages/Cheesy Chicken Ranch Sandwiches.jpg
./src/recipeImages/Cottage Cheese with an Attitude.jpg
made it
//assets.kraftfoods.com/recipe_images/Cottage_Cheese_with_An_Attitude.jpg
./src/recipeImages/Grilled Cheese and Vegetable Baguette.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/118842_640x428.jpg
./src/recipeImages/Shrimp & Cucumber Noodle Salad.jpg
made it
//assets.kraftfoods.com/reci

./src/recipeImages/Super Smoothie.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/118214_640x428.jpg
./src/recipeImages/No-Drip Strawberry Ice Pops.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/117984_640x428.jpg
./src/recipeImages/KOOL-AID Float.jpg
made it
//assets.kraftfoods.com/recipe_images/KOOL-AID_Float.jpg
./src/recipeImages/Peppermint Hot Chocolate.jpg
made it
//assets.kraftfoods.com/recipe_images/Peppermint-Hot-Chocolate-52825.jpg
./src/recipeImages/Cinnamon-Caramel Iced Coffee Recipe.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/52782_SGS_K2836V0_OR1_OH_640x428.jpg
./src/recipeImages/Sangria Party Cups.jpg
made it
//assets.kraftfoods.com/recipe_images/Sangria_Party_Cups.jpg
./src/recipeImages/Strawberry Swirl Shake.jpg
made it
//assets.kraftfoods.com/recipe_images/Strawberry_Swirl_Shake.jpg
./src/recipeImages/Chocolate-Hazelnut Iced Latte Recipe.jpg
made it
//assets.kraftfoods.com/recipe_images/Iced-Chocolate-Hazelnut-Latte-63847.jp

./src/recipeImages/Cheese-Topped Grilled Tomatoes.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/106266.tif?id=e6ca0a6d0cc6dae00bef25f718f1e62620869734&ht=225&wd=225&version=1&clid=pim&yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/106266.tif?id=e6ca0a6d0cc6dae00bef25f718f1e62620869734&ht=225&wd=225&version=1&clid=pim&yocs=1b_
./src/recipeImages/Cheese-Topped Grilled Tomatoes.jpg
./src/recipeImages/Mediterranean Chicken and Mushroom Skillet.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/123036_640x428.jpg
On index 160
./src/recipeImages/Mediterranean Pork Medallions.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/104576_640x428.jpg
./src/recipeImages/Mediterranean Chicken Recipe.jpg
made it
//assets.kraftfoods.

...

./src/recipeImages/Praline Sweet Potatoes.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/54242_SGS_K2907V0_OR1_CR_640x428.jpg
./src/recipeImages/Old-Fashioned Succotash Recipe.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/90613.tif?id=0723fb817545bbe32cfd689b41c6808dd0074922&ht=225&wd=225&version=1&clid=pim&yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/90613.tif?id=0723fb817545bbe32cfd689b41c6808dd0074922&ht=225&wd=225&version=1&clid=pim&yocs=1b_
./src/recipeImages/Old-Fashioned Succotash Recipe.jpg
./src/recipeImages/Chicken-Fried Steak with Beef Gravy.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendit

./src/recipeImages/Hawaiian BBQ Glazed Pork Chops.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/2de6dbe02d150137fdf1123dfe2baf36.yottaa.net/v~4b.179/kraftrecipes/v201911010408/img/noImage.png?yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/2de6dbe02d150137fdf1123dfe2baf36.yottaa.net/v~4b.179/kraftrecipes/v201911010408/img/noImage.png?yocs=1b_
./src/recipeImages/Hawaiian BBQ Glazed Pork Chops.jpg
./src/recipeImages/Hawaiian Coffee for Two.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/2de6dbe02d150137fdf1123dfe2baf36.yottaa.net/v~4b.179/kraftrecipes/v201911010408/img/noImage.png?yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/2de6dbe02d150137fdf1123dfe2baf36.yottaa.net/v~4b.179/kraftrecipes/v201911010408/img/noImage.png?yocs=1b_
./src/recipeImages/Hawaiian Coffee for Two.jpg
./src/recipeImages/Hawaiian Cookies.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/2de6dbe02d150137fdf1123dfe2baf36.yottaa.net/v~4b.

./src/recipeImages/Coconut-Curried Shrimp with Chutney Mayo.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/115669_640x428.jpg
./src/recipeImages/Flank Steak with Black Bean Salsa.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/109730_640x428.jpg
./src/recipeImages/Fried Plantains Recipe.jpg
made it
//assets.kraftfoods.com/recipe_images/Fried_Plantains_with_Sweet_Cream_Sauce1.jpg
./src/recipeImages/Grilled Fruit Salad.jpg
made it
//assets.kraftfoods.com/recipe_images/Grilled_Fruit_Salad1.jpg
./src/recipeImages/Layered Island Dip.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/Layered-Island-Dip-57514_640x428.jpg
./src/recipeImages/Caribbean Cooler.jpg
made it
//assets.kraftfoods.com/recipe_images/Caribbean_Cooler.jpg
./src/recipeImages/Key Lime Margarita Pie.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/75164_640x428.jpg
./src/recipeImages/Jerk Chicken.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/52899_SGS_K164%20%20

./src/recipeImages/Brined Sage Turkey with Mushroom Stuffing.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/120329_640x428.jpg
./src/recipeImages/Cranberry-Orange White Chocolate Cheesecake.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/199805_H1.jpg?id=d5c2a9d5699474dac95168180241b68ec58eef72&ht=225&wd=225&clid=pim&yocs=1b_
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/199805_H1.jpg?id=d5c2a9d5699474dac95168180241b68ec58eef72&ht=225&wd=225&clid=pim&yocs=1b_
./src/recipeImages/Cranberry-Orange White Chocolate Cheesecake.jpg
./src/recipeImages/Easy Corn Pudding Casserole.jpg
made it
https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia

./src/recipeImages/Crispy Football-Shaped Treats.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/166472_640x428.jpg
./src/recipeImages/Famous Sub Shop Club.jpg
./src/recipeImages/Flank Steak with Black Bean Salsa.jpg
./src/recipeImages/Inside-Out Cheeseburger.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/144633_640x428.jpg
./src/recipeImages/Baby Shower Cake.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/148789_640x428.jpg
On index 400
./src/recipeImages/Buccaneer Cake.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/113961_640x428.jpg
./src/recipeImages/Butterfly Cake.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/132977_640x428.jpg
./src/recipeImages/Carousel Cake.jpg
made it
//assets.kraftfoods.com/recipe_images/Merry-Go-Round_Cake.jpg
./src/recipeImages/Fabulous Tropical Fruit Salad.jpg
./src/recipeImages/Greek Spinach Dip.jpg
made it
//assets.kraftfoods.com/recipe_images/opendeploy/128453_640x428.jpg
./src/recipeI

In [40]:
downloadLocation

'.src/recipeImages/Baked Crab Rangoon.jpg'

In [ ]:
'./src/recipeImages/test.jpg'

In [43]:
wget.download(imageLink, './src/recipeImages/Baked Crab Rangoon.jpg')

'./src/recipeImages/Baked Crab Rangoon.jpg'

In [48]:
data[2]

('dinner',
 'Dinner Recipes',
 'https://www.myfoodandfamily.com//recipe/146260/hearty-italian-sausage-soup?categoryid=3',
 'https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/HeartyItallianSSausageSoup_01.jpg?id=375b0eaa2fda971dfbc16075522e252b8401eef7&ht=225&wd=225&yocs=1b_')

In [ ]:
aa

In [159]:
df = pd.DataFrame(totalData)

In [160]:
# Create random list of datetimes

In [161]:
df.head()

,category,description,estimatedTime,imageName,ingredients,procedure,subcategory,title
0,dinner,Learn how to make these crispy Baked Crab Rang...,40 Min(s),Baked Crab Rangoon.jpg,[4 oz. (1/2 of 8-oz. pkg.) PHILADELPHIA Neufch...,"[Heat oven to 350°F., Mix first 4 ingredients ...",Dinner Recipes,Baked Crab Rangoon
1,dinner,Enjoy a devilishly good Country-Style Smashed ...,35 Min(s),Country-Style Smashed Potato Salad.jpg,"[3 lb. Yukon gold potatoes (about 6 large), pe...",[Cook potatoes in boiling water in saucepan 10...,Dinner Recipes,Country-Style Smashed Potato Salad
2,dinner,Enjoy a bowl of our Hearty Italian Sausage Sou...,40 Min(s),Hearty Italian Sausage Soup.jpg,[1 pkg. (13 oz.) OSCAR MAYER Natural Uncured I...,[Cook sausage and onions in large skillet on m...,Dinner Recipes,Hearty Italian Sausage Soup
3,dinner,Savor Lentil Soup during cold months or year-r...,50 Min(s),Lentil Soup.jpg,"[6 slices OSCAR MAYER Bacon, chopped, 1 small ...",[Cook bacon and onions in large saucepan 8 to ...,Dinner Recipes,Lentil Soup
4,dinner,Trying to find a speedy beef stir-fry that'll ...,25 Min(s),Speedy Barbecue Beef Stir-Fry.jpg,"[1/4 cup KRAFT Original Barbecue Sauce, 3 Tbsp...",[Mix barbecue sauce and broth until blended; t...,Dinner Recipes,Speedy Barbecue Beef Stir-Fry


In [163]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [164]:
start = datetime.strptime('1/1/2018 12:00 AM', '%m/%d/%Y %I:%M %p')
end = datetime.strptime('11/1/2019 12:00 AM', '%m/%d/%Y %I:%M %p')

In [165]:
times = []
for i in range(len(df)):
    dates = []
    count = randint(1, 5)
    if count < 3:
        count = 1
    else:
        count -= 1
    for i in range(count):
        randomDate = random_date(start, end)
        randomDate = randomDate.strftime("%m/%d/%Y, %H:%M:%S")
        dates.append(randomDate)
    dates.sort()
    times.append(dates)

In [166]:
df['times'] = times

In [167]:
# Do favorites
isFavorite = [True if randint(1,4) == 4 else False for i in range(len(df))]
df['isFavorite'] = isFavorite

In [187]:
# Convert estimatedTime to minutes
convertedTimes = []
times = df['estimatedTime'].values
for time in times:
    if time is None:
        convertedTimes.append(None)
    else:
        numbers = re.findall(r'[0-9]+', time) 
        if len(numbers) == 2:
            total = int(numbers[1]) + int(numbers[0]) * 60
        elif len(numbers) == 1:
            total = int(numbers[0])
        else:
            print('error')
        convertedTimes.append(total)

In [191]:
df['estimatedTime'] = convertedTimes

In [193]:
# Save to csv
df.to_csv('data.csv', index = False)

In [195]:
# Change to json
jsonData = df.to_json(orient = 'records')

In [196]:
with open('jsonData.json', 'w') as json_file:
    json.dump(jsonData, json_file)